In [97]:
from arff2pandas import a2p
import pandas as pd
import numpy as np


In [98]:
with open('jm1.arff') as f:
    df = a2p.load(f)
    print(df)

       loc@NUMERIC  v(g)@NUMERIC  ev(g)@NUMERIC  iv(g)@NUMERIC  n@NUMERIC  \
0              1.1           1.4            1.4            1.4        1.3   
1              1.0           1.0            1.0            1.0        1.0   
2             72.0           7.0            1.0            6.0      198.0   
3            190.0           3.0            1.0            3.0      600.0   
4             37.0           4.0            1.0            4.0      126.0   
5             31.0           2.0            1.0            2.0      111.0   
6             78.0           9.0            5.0            4.0        0.0   
7              8.0           1.0            1.0            1.0       16.0   
8             24.0           2.0            1.0            2.0        0.0   
9            143.0          22.0           20.0           10.0        0.0   
10            73.0          10.0            4.0            6.0        0.0   
11            83.0          11.0           10.0            7.0        0.0   

Base com missing values, preencher com a media

In [99]:
df = df.interpolate()

In [100]:
input_features = df.drop(["defects@{false,true}"], axis=1)

Base desbalanceada -> lembrar de pegar k-fold estratificado

In [53]:
output_class = df["defects@{false,true}"]
len(output_class[output_class == "true"])

8779

In [101]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

In [55]:
#Bagging
es = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                       max_samples=0.80, max_features=1.0, bootstrap=True, 
                       bootstrap_features=False, n_jobs=3)

In [58]:
np.mean(cross_val_score(es, input_features,output_class, cv=10 ,scoring='accuracy'))

0.7941275756845274

In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_features, output_class,test_size=0.2, random_state=42)

In [88]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron

In [87]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
probs = clf.predict_proba(X_test)

In [90]:
clf = Perceptron()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
#probs = clf.predict_proba(X_test)

/home/andrea/Workspace/mlproject/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [92]:
accuracy_score(y_test, pred)

0.8075333027101516

In [103]:
from sklearn.ensemble import VotingClassifier

In [113]:
es2 = VotingClassifier([('dt', DecisionTreeClassifier()), ('p', Perceptron())], voting='soft')

In [114]:
es2.fit(X_train, y_train)

/home/andrea/Workspace/mlproject/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


VotingClassifier(estimators=[('dt', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf...n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [116]:
pred = es2.predict(X_test)

AttributeError: 'Perceptron' object has no attribute 'predict_proba'

In [112]:
accuracy_score(y_test, pred)

0.8075333027101516